# 🫀 Clinical-Grade PTB-XL ECG Preprocessing Pipeline  
<span style="color:red">by Ridwan Oladipo, MD | Medical AI Specialist</span>  

Production-grade preprocessing pipeline for 12-lead ECG classification, transforming the **PTB-XL dataset** (~21,837 recordings) with:  

- **Clinical diagnostic mapping** (SCP codes → 5 superclasses: NORM, MI, STTC, CD, HYP)  
- **Cardiology-informed feature extraction** (heart rate, HRV, ST-segment analysis, T-wave morphology)  
- **Multi-lead statistical profiling** (time/frequency domain features across all 12 leads)  
- **Myocardial infarction anatomical localization** (anterior/inferior/lateral subtype classification)  
- **Signal standardization** (10-second, 100Hz, 12-lead arrays with batch processing)  
- **Stratified data splitting** (preserving diagnostic distributions, preventing patient leakage)  

🚀 Processes **21,837 ECG recordings** from **18,885 patients** into comprehensive **signal + tabular feature matrices** with 200+ engineered features per sample, ready for multimodal deep learning.  

>⚕️ **Clinical expertise meets scalable preprocessing** — bridging cardiology practice with enterprise-grade data pipelines.

## 🧩 Environment Setup

In [1]:
# Install the WFDB package for ECG signal processing
!pip install wfdb

In [2]:
# Essential libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import wfdb
import ast
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import scipy.stats
from scipy.signal import find_peaks, welch
from scipy.fft import fft
import time
from tqdm import tqdm

# For visualization
import plotly.express as px
import plotly.graph_objects as go

# Set professional styling for visualizations
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.family'] = 'sans-serif'

print("Environment setup complete.")
print("Libraries imported successfully.")

Environment setup complete.
Libraries imported successfully.


## 📊 Dataset Loading and Initial Exploration

In [3]:
# Define the data directory
data_dir = "/kaggle/input/ptbxlphysionet/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1"

# Load the metadata
df_meta = pd.read_csv(os.path.join(data_dir, 'ptbxl_database.csv'), index_col='ecg_id')

# Parse SCP codes from string to dictionary format
df_meta['scp_codes'] = df_meta['scp_codes'].apply(lambda x: ast.literal_eval(x))

# Load the SCP-ECG statement descriptions for diagnostic mapping
scp_statements = pd.read_csv(os.path.join(data_dir, 'scp_statements.csv'), index_col=0)

# Dataset overview
print("=== PTB-XL Dataset Overview ===")
print(f"Total number of ECG recordings: {len(df_meta):,}")
print(f"Patient age range: {df_meta['age'].min():.0f} to {df_meta['age'].max():.0f} years")
print(f"Gender distribution:")
gender_counts = df_meta['sex'].value_counts()
for sex, count in gender_counts.items():
    gender_label = 'Female' if sex == 0 else 'Male'
    print(f"  {gender_label}: {count:,} ({count / len(df_meta) * 100:.1f}%)")

print(f"\nDiagnostic classes available in SCP statements:")
print(scp_statements['diagnostic_class'].value_counts())

# Check data quality
missing_values = df_meta.isnull().sum()
print(f"\nData Quality Assessment:")
print(f"Missing age values: {df_meta['age'].isna().sum():,} ({df_meta['age'].isna().sum() / len(df_meta) * 100:.2f}%)")

# Fill missing age values with median (clinical standard approach)
df_meta['age'] = df_meta['age'].fillna(df_meta['age'].median())
print(f"Age values imputed with median: {df_meta['age'].median():.1f} years")

=== PTB-XL Dataset Overview ===
Total number of ECG recordings: 21,837
Patient age range: 2 to 95 years
Gender distribution:
  Female: 11,379 (52.1%)
  Male: 10,458 (47.9%)

Diagnostic classes available in SCP statements:
diagnostic_class
MI      14
STTC    13
CD      11
HYP      5
NORM     1
Name: count, dtype: int64

Data Quality Assessment:
Missing age values: 89 (0.41%)
Age values imputed with median: 62.0 years


## 🏥 Clinical Diagnostic Mapping

In [4]:
def aggregate_diagnostic(y_dic):
    """
    Map SCP codes to primary diagnostic classes following clinical guidelines
    
    Args:
        y_dic: Dictionary of SCP codes with confidence scores
        
    Returns:
        List of primary diagnostic classes
    """
    tmp = []
    for key in y_dic.keys():
        if key in scp_statements.index:
            diagnostic_class = scp_statements.loc[key].diagnostic_class
            if not pd.isna(diagnostic_class):
                tmp.append(diagnostic_class)
    return list(set(tmp))

# Apply diagnostic mapping
df_meta['diagnostic_superclass'] = df_meta['scp_codes'].apply(aggregate_diagnostic)

# Analyze diagnostic class distribution
class_counts = {}
for classes in df_meta['diagnostic_superclass']:
    for cls in classes:
        class_counts[cls] = class_counts.get(cls, 0) + 1

print("=== Primary Diagnostic Class Distribution ===")
for cls, count in sorted(class_counts.items(), key=lambda x: x[1], reverse=True):
    percentage = count / len(df_meta) * 100
    print(f"{cls}: {count:,} records ({percentage:.1f}%)")

# Create target labels for multi-label classification
def create_target_labels(df):
    """Create one-hot encoded target labels for the five primary classes"""
    result_df = df.copy()
    
    # Initialize columns for each diagnostic class
    for cls in ['NORM', 'MI', 'STTC', 'CD', 'HYP']:
        result_df.loc[:, cls] = result_df['diagnostic_superclass'].apply(lambda x: 1 if cls in x else 0)
    
    return result_df

# Apply target label creation
df_meta_with_labels = create_target_labels(df_meta)

print(f"\n=== Target Label Distribution ===")
for cls in ['NORM', 'MI', 'STTC', 'CD', 'HYP']:
    count = df_meta_with_labels[cls].sum()
    percentage = count / len(df_meta_with_labels) * 100
    print(f"{cls}: {count:,} cases ({percentage:.1f}%)")

# Check for multi-label cases (clinically realistic)
multi_label_count = sum(1 for classes in df_meta_with_labels['diagnostic_superclass'] if len(classes) > 1)
print(f"\nMulti-label cases: {multi_label_count:,} ({multi_label_count / len(df_meta_with_labels) * 100:.1f}%)")
print("Note: Multi-label cases reflect clinical reality where patients may have multiple cardiac conditions.")

=== Primary Diagnostic Class Distribution ===
NORM: 9,528 records (43.6%)
MI: 5,486 records (25.1%)
STTC: 5,250 records (24.0%)
CD: 4,907 records (22.5%)
HYP: 2,655 records (12.2%)

=== Target Label Distribution ===
NORM: 9,528 cases (43.6%)
MI: 5,486 cases (25.1%)
STTC: 5,250 cases (24.0%)
CD: 4,907 cases (22.5%)
HYP: 2,655 cases (12.2%)

Multi-label cases: 5,158 (23.6%)
Note: Multi-label cases reflect clinical reality where patients may have multiple cardiac conditions.


#### *Note: The most common code is* **SR (Sinus Rhythm)**. *This is a rhythm annotation, not a diagnostic subclass, so its diagnostic_class is NaN and it is not included in the five diagnostic categories (NORM, MI, STTC, CD, HYP) used for modeling.*